In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 1: Load and preprocess your dataset
# data = pd.read_csv('')  # Update with your dataset file path
# import dataset
pd.set_option('display.max_colwidth', 100)
data =pd.read_csv("D:\pythonana\Dataset-SA.csv")
# Extract the text and label columns


In [3]:
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Download the stop words corpus
# nltk.download('stopwords')

stops = set(stopwords.words('english')) #english stopwords

stemmer = SnowballStemmer('english') #SnowballStemmer

def review_to_words(raw_review):
    # 1. Delete HTML 
    try:
        review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
        # 2. Make a space
        letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. lower letters
        words = letters_only.lower().split()
        # 5. Stopwords 
        meaningful_words = [w for w in words if not w in stops]
        # 6. Stemming
        stemming_words = [stemmer.stem(w) for w in meaningful_words]
        # 7. space join words
        return( ' '.join(stemming_words))
    except:
        return "i"

In [4]:
data['Summary'] = data['Summary'].apply(review_to_words)

In [5]:
texts = data['Summary'].tolist()

labels = data['Sentiment'].tolist()

# Step 2: Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 3: Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts)

# Step 4: Pad sequences to have equal length
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len)

# Step 5: Split your dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# Step 6: Build the CNN model
embedding_dim = 100
filters = 64
kernel_size = 5

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Step 7: Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

# Step 8: Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Step 9: Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(y_pred)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 65, 100)           3548000   
                                                                 
 conv1d (Conv1D)             (None, 61, 64)            32064     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                        

In [25]:
# Preprocess the incoming data
incoming_text = "not good"
incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# Predict the sentiment label for the incoming data
predicted_label = np.argmax(model.predict(incoming_padded_sequence), axis=-1)

# Decode the predicted label
decoded_label = label_encoder.inverse_transform(predicted_label)[0]

print("Incoming Text:", incoming_text)
print("Predicted Label:", decoded_label)


1/1 [==============================] - 0s 61ms/step
Incoming Text: not good
Predicted Label: positive


In [16]:
# import tensorflow as tf

# # Save the model
# # tf.saved_model.save(model, 'saved_model')
# tf.keras.models.save_model(model, 'saved_model')

# # Load the model
# # loaded_model = tf.saved_model.load('saved_model')

# import tensorflow as tf

# # Load the saved model
# loaded_model = tf.keras.models.load_model('saved_model')

# # Now you can use the loaded model to make predictions on new data
# incoming_text = "value for money"
# incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
# incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# # Predict the sentiment label for the incoming data
# predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

# # Decode the predicted label
# decoded_label = label_encoder.inverse_transform(predicted_label)[0]

# print("Incoming Text:", incoming_text)
# print("Predicted Label:", decoded_label)



INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


1/1 [==============================] - 0s 192ms/step
Incoming Text: value for money
Predicted Label: positive


In [9]:
# import tensorflow as tf

# # Load the saved model
# loaded_model = tf.saved_model.load('saved_model')

# # Preprocess the incoming data
# # incoming_text = "value for money"
# # Perform any necessary preprocessing on the incoming text
# incoming_text = "not bad"
# incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
# incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# # Make predictions on the incoming data
# predictions = loaded_model(incoming_padded_sequence)


ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * <tf.Tensor 'inputs:0' shape=(1, 65) dtype=int32>
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 65), dtype=tf.float32, name='embedding_input')
    * True
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 65), dtype=tf.float32, name='embedding_input')
    * False
    * None
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 65), dtype=tf.float32, name='inputs')
    * True
    * None
  Keyword arguments: {}

Option 4:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 65), dtype=tf.float32, name='inputs')
    * False
    * None
  Keyword arguments: {}

In [15]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer

# # Load the saved model
# loaded_model = tf.keras.models.load_model('saved_model')

# # Preprocess the incoming data
# incoming_text = "worst"

# # Tokenize the incoming text
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts([incoming_text])
# sequences = tokenizer.texts_to_sequences([incoming_text])

# # Pad the sequences to have the same length as the training data
# max_len = 65  # Maximum sequence length used during training
# padded_sequences = pad_sequences(sequences, maxlen=max_len)

# # Make predictions on the preprocessed data
# predictions = loaded_model.predict(padded_sequences)

# # Decode the predicted labels
# predicted_label = np.argmax(predictions, axis=1)
# decoded_label = label_encoder.inverse_transform(predicted_label)[0]

# print("Incoming Text:", incoming_text)
# print("Predicted Label:", decoded_label)


1/1 [==============================] - 0s 185ms/step
Incoming Text: worst
Predicted Label: positive


In [19]:
# import tensorflow as tf

# # Load the saved model
# loaded_model = tf.keras.models.load_model('saved_model')

# # Now you can use the loaded model to make predictions on new data
# incoming_text = "useless"
# incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
# incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# # Predict the sentiment label for the incoming data
# predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

# # Decode the predicted label
# decoded_label = label_encoder.inverse_transform(predicted_label)[0]

# print("Incoming Text:", incoming_text)
# print("Predicted Label:", decoded_label)



1/1 [==============================] - 0s 178ms/step
Incoming Text: useless
Predicted Label: positive
